In [29]:
import numba
print(numba.__version__)

0.35.0+10.g143f70e


In [30]:
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [31]:
import numpy as np

original = np.arange(0.0, 10.0, 0.01, dtype='f4')
shuffled = original.copy()
np.random.shuffle(shuffled)

In [32]:
sorted = shuffled.copy()
bubblesort(sorted)
print(np.array_equal(sorted, original))

True


In [33]:
sorted[:] = shuffled[:]
%timeit sorted[:] = shuffled[:]; bubblesort(sorted)

141 ms ± 3.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
print(numba.jit.__doc__)


    This decorator is used to compile a Python function into native code.

    Args
    -----
    signature:
        The (optional) signature or list of signatures to be compiled.
        If not passed, required signatures will be compiled when the
        decorated function is called, depending on the argument values.
        As a convenience, you can directly pass the function to be compiled
        instead.

    locals: dict
        Mapping of local variable names to Numba types. Used to override the
        types deduced by Numba's type inference engine.

    target: str
        Specifies the target platform to compile for. Valid targets are cpu,
        gpu, npyufunc, and cuda. Defaults to cpu.

    options:
        For a cpu target, valid options are:
            nopython: bool
                Set to True to disable the use of PyObjects and Python API
                calls. The default behavior is to allow the use of PyObjects
                and Python API. Default value is Fal

In [35]:
bubblesort_jit = numba.jit("void(f4[:])")(bubblesort)

In [36]:
sorted[:] = shuffled[:] # reset to shuffled before sorting
bubblesort_jit(sorted)
print(np.array_equal(sorted, original))

True


In [37]:
%timeit sorted[:] = shuffled[:]; bubblesort_jit(sorted)

571 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
%timeit sorted[:] = shuffled[:]; bubblesort(sorted)

137 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
@numba.jit("void(f4[:])")
def bubblesort_jit(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [40]:
bubblesort_autojit = numba.jit(bubblesort)

In [41]:
%timeit sorted[:] = shuffled[:]; bubblesort_autojit(sorted)

988 µs ± 2.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [62]:
import sys
@numba.jit("void(i1[:])")
def test(value):
    for i in range(len(value)):
        value[i] = i % 100

from decimal import Decimal
@numba.jit("void(i1[:])")
def test2(value):
    for i in range(len(value)):
        value[i] = i % Decimal(100)

res = np.zeros((10000,), dtype="i1")

**Note :** xrange was in python 2. In python 3 it was renamed to range. [ref](https://stackoverflow.com/questions/17192158/nameerror-global-name-xrange-is-not-defined-in-python-3)

In [56]:
%timeit test(res)

12.3 µs ± 72.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [57]:
%timeit test2(res)

3.86 ms ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [59]:
@numba.jit("void(i1[:])", nopython=True)
def test(value):
    for i in range(len(value)):
        value[i] = i % 100

In [61]:
@numba.jit("void(i1[:])", nopython=True)
def test2(value):
    for i in range(len(value)):
        value[i] = i % Decimal(100)

TypingError: Failed at nopython (nopython frontend)
Untyped global name 'Decimal': cannot determine Numba type of <class 'type'>
File "<ipython-input-61-5fae05749356>", line 4